In [13]:
import os
import numpy as np
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import cv2
import re
# import os
import glob
# import re
# import numpy as np
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
import tifffile as tiff
# import os
# import numpy as np
# import glob
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
# import rasterio

def load_image(filepath):
    # Load the image using PIL
    image = np.array(Image.open(filepath).convert('L'))
    
    # Apply Otsu's thresholding
    t, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Convert to binary (0 and 1)
    binary_image = (binary_image > 0).astype(np.uint8)
    return binary_image

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)

# Paths to the folders containing the predictions and ground truths


In [14]:
# gt_folder = './GEE_Masks/GEE_resized/'
# pred_folder = './GEE_Output/Adversarial'

# # List all files in the prediction folder
# # pred_files = [f for f in os.listdir(pred_folder) if f.endswith('.tif')]

# # Initialize metrics



In [15]:
# # Lists to store metrics
# dice_scores = []
# iou_scores = []
# pixel_accuracies = []
# precisions = []
# recalls = []
# f1_scores = []
# specificities = []

# for i in range(0, 1263):  # Adjust the range based on the number of files
#     gt_filename = f'NDWI_Mask_{i}_resized.tif'
#     pred_filename = f'dense_Predicted_Image_{i}.tif'
    
#     gt_path = os.path.join(gt_folder, gt_filename)
#     pred_path = os.path.join(pred_folder, pred_filename)
    
#     # Load the prediction and ground truth images
#     pred_image = load_image(pred_path)
#     gt_image = load_image(gt_path)
    
#     # Flatten the images
#     pred_flat = pred_image.flatten()
#     gt_flat = gt_image.flatten()
    
#     # Calculate metrics
#     dice = dice_coefficient(gt_flat, pred_flat)
#     iou = jaccard_score(gt_flat, pred_flat, average='macro')
#     accuracy = pixel_accuracy(gt_flat, pred_flat)
#     precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
#     recall = recall_score(gt_flat, pred_flat, average='macro')
#     f1 = f1_score(gt_flat, pred_flat, average='macro')
#     spec = specificity(gt_flat, pred_flat)
    
#     # Store metrics
#     dice_scores.append(dice)
#     iou_scores.append(iou)
#     pixel_accuracies.append(accuracy)
#     precisions.append(precision)
#     recalls.append(recall)
#     f1_scores.append(f1)
#     specificities.append(spec)
    
#     # Print metrics for the current image
#     print(f"Metrics for {gt_filename} and {pred_filename}:")
#     print(f"  Dice Coefficient: {dice:.4f}")
#     print(f"  IoU: {iou:.4f}")
#     print(f"  Pixel Accuracy: {accuracy:.4f}")
#     print(f"  Precision: {precision:.4f}")
#     print(f"  Recall: {recall:.4f}")
#     print(f"  F1 Score: {f1:.4f}")
#     print(f"  Specificity: {spec:.4f}")
#     print()

# # Calculate and print average metrics
# average_dice = np.mean(dice_scores)
# average_iou = np.mean(iou_scores)
# average_accuracy = np.mean(pixel_accuracies)
# average_precision = np.mean(precisions)
# average_recall = np.mean(recalls)
# average_f1 = np.mean(f1_scores)
# average_specificity = np.mean(specificities)

# print("Average metrics:")
# print(f"  Average Dice Coefficient: {average_dice:.4f}")
# print(f"  Average IoU: {average_iou:.4f}")
# print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
# print(f"  Average Precision: {average_precision:.4f}")
# print(f"  Average Recall: {average_recall:.4f}")
# print(f"  Average F1 Score: {average_f1:.4f}")
# print(f"  Average Specificity: {average_specificity:.4f}")

# # Model's overall accuracy
# print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

In [16]:
# # Print average metrics
# print("Average Metrics:")
# print(f"  Average Dice Coefficient: {np.mean(dice_scores):.4f}")
# print(f"  Average IoU: {np.mean(iou_scores):.4f}")
# print(f"  Average Pixel Accuracy: {np.mean(pixel_accuracies):.4f}")
# print(f"  Average Precision: {np.mean(precisions):.4f}")
# print(f"  Average Recall: {np.mean(recalls):.4f}")
# print(f"  Average F1 Score: {np.mean(f1_scores):.4f}")
# print(f"  Average Specificity: {np.mean(specificities):.4f}")

In [17]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)


# Directories for ground truth and prediction masks
gt_folder = './GEE_Masks/GEE_resized/'
pred_folder = './GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels'

# Initialize metric lists
dice_scores = []
iou_scores = []
pixel_accuracies = []
precisions = []
recalls = []
f1_scores = []
specificities = []

# Get all prediction files
pred_files = glob.glob(os.path.join(pred_folder, 'dense_*.tif'))

for pred_file in pred_files:
    # Extract the number 'i' from the prediction filename
    match = re.search(r'dense_(\d+).tif', os.path.basename(pred_file))
    if match:
        i = match.group(1)
        gt_filename = f'NDWI_Mask_{i}_resized.tif'
        gt_path = os.path.join(gt_folder, gt_filename)
        
        # Check if the ground truth file exists
        if os.path.exists(gt_path):
            # Load the prediction and ground truth images
            pred_image = load_image(pred_file)
            gt_image = load_image(gt_path)
            print(np.max(pred_image))
            print(np.min(pred_image))
            # Flatten the images
            pred_flat = pred_image.flatten()
            gt_flat = gt_image.flatten()
            
            # Calculate metrics
            dice = dice_coefficient(gt_flat, pred_flat)
            iou = jaccard_score(gt_flat, pred_flat, average='macro')
            accuracy = pixel_accuracy(gt_flat, pred_flat)
            precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
            recall = recall_score(gt_flat, pred_flat, average='macro')
            f1 = f1_score(gt_flat, pred_flat, average='macro')
            spec = specificity(gt_flat, pred_flat)
            
            # Store metrics
            dice_scores.append(dice)
            iou_scores.append(iou)
            pixel_accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
            specificities.append(spec)
            
            # Print metrics for the current image
            print(f"Metrics for {gt_filename} and {pred_file}:")
            print(f"  Dice Coefficient: {dice:.4f}")
            print(f"  IoU: {iou:.4f}")
            print(f"  Pixel Accuracy: {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1 Score: {f1:.4f}")
            print(f"  Specificity: {spec:.4f}")
            print()
        else:
            print(f"Ground truth file {gt_path} not found for prediction file {pred_file}.")

# Calculate and print average metrics
average_dice = np.mean(dice_scores)
average_iou = np.mean(iou_scores)
average_accuracy = np.mean(pixel_accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)
average_specificity = np.mean(specificities)

print("Average Metrics:")
print(f"  Dice Coefficient: {average_dice:.4f}")
print(f"  IoU: {average_iou:.4f}")
print(f"  Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Precision: {average_precision:.4f}")
print(f"  Recall: {average_recall:.4f}")
print(f"  F1 Score: {average_f1:.4f}")
print(f"  Specificity: {average_specificity:.4f}")

1
0
Metrics for NDWI_Mask_10_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_10.tif:
  Dice Coefficient: 0.0719
  IoU: 0.3257
  Pixel Accuracy: 0.6197
  Precision: 0.7998
  Recall: 0.5185
  F1 Score: 0.4164
  Specificity: 0.9996

1
0
Metrics for NDWI_Mask_1008_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1008.tif:
  Dice Coefficient: 0.0670
  IoU: 0.4779
  Pixel Accuracy: 0.9213
  Precision: 0.8513
  Recall: 0.5171
  F1 Score: 0.5130
  Specificity: 0.9991

1
0
Metrics for NDWI_Mask_1010_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1010.tif:
  Dice Coefficient: 0.0280
  IoU: 0.3388
  Pixel Accuracy: 0.6650
  Precision: 0.8017
  Recall: 0.5069
  F1 Score: 0.4128
  Specificity: 0.9995

1
0
Metrics for NDWI_Mask_1012_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1012.tif:
  Dice Coefficient: 0.1011
  IoU: 0.4962
  Pixel Accuracy: 0.9394
  Precision: 0.

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_1079_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1079.tif:
  Dice Coefficient: 0.0172
  IoU: 0.4999
  Pixel Accuracy: 0.9998
  Precision: 0.5000
  Recall: 0.4999
  F1 Score: 0.4999
  Specificity: 0.9998

1
0
Metrics for NDWI_Mask_1086_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1086.tif:
  Dice Coefficient: 0.0061
  IoU: 0.4997
  Pixel Accuracy: 0.9994
  Precision: 0.4999
  Recall: 0.4998
  F1 Score: 0.4998
  Specificity: 0.9996

1
0
Metrics for NDWI_Mask_1094_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1094.tif:
  Dice Coefficient: 0.0519
  IoU: 0.4841
  Pixel Accuracy: 0.9416
  Precision: 0.8752
  Recall: 0.5132
  F1 Score: 0.5108
  Specificity: 0.9996

1
0
Metrics for NDWI_Mask_1103_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1103.tif:
  Dice Coefficient: 0.0092
  IoU: 0.5009
  Pixel Accuracy: 0.9988
  Precision: 0.

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_1149_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1149.tif:
  Dice Coefficient: 0.0088
  IoU: 0.4998
  Pixel Accuracy: 0.9996
  Precision: 0.5000
  Recall: 0.4998
  F1 Score: 0.4999
  Specificity: 0.9996

1
0
Metrics for NDWI_Mask_1160_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1160.tif:
  Dice Coefficient: 0.0972
  IoU: 0.4974
  Pixel Accuracy: 0.9439
  Precision: 0.8612
  Recall: 0.5254
  F1 Score: 0.5341
  Specificity: 0.9991

1
0
Metrics for NDWI_Mask_1161_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1161.tif:
  Dice Coefficient: 0.0153
  IoU: 0.3318
  Pixel Accuracy: 0.6567
  Precision: 0.7343
  Recall: 0.5034
  F1 Score: 0.4037
  Specificity: 0.9991

1
0
Metrics for NDWI_Mask_1164_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_1164.tif:
  Dice Coefficient: 0.0188
  IoU: 0.3479
  Pixel Accuracy: 0.6873
  Precision: 0.

...

Metrics for NDWI_Mask_930_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_930.tif:
  Dice Coefficient: 0.0066
  IoU: 0.4784
  Pixel Accuracy: 0.9535
  Precision: 0.5845
  Recall: 0.5014
  F1 Score: 0.4913
  Specificity: 0.9994

1
0
Metrics for NDWI_Mask_931_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_931.tif:
  Dice Coefficient: 0.0655
  IoU: 0.4107
  Pixel Accuracy: 0.7890
  Precision: 0.8563
  Recall: 0.5166
  F1 Score: 0.4733
  Specificity: 0.9992

1
0
Metrics for NDWI_Mask_938_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_938.tif:
  Dice Coefficient: 0.0001
  IoU: 0.4853
  Pixel Accuracy: 0.9705
  Precision: 0.4854
  Recall: 0.4998
  F1 Score: 0.4925
  Specificity: 0.9996

1
0
Metrics for NDWI_Mask_941_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_941.tif:
  Dice Coefficient: 0.0973
  IoU: 0.4816
  Pixel Accuracy: 0.9125
  Precision: 0.8937
  R

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_948_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_948.tif:
  Dice Coefficient: 0.0133
  IoU: 0.4999
  Pixel Accuracy: 0.9997
  Precision: 0.5000
  Recall: 0.4999
  F1 Score: 0.4999
  Specificity: 0.9997

1
0
Metrics for NDWI_Mask_95_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_95.tif:
  Dice Coefficient: 0.0093
  IoU: 0.4998
  Pixel Accuracy: 0.9996
  Precision: 0.5000
  Recall: 0.4998
  F1 Score: 0.4999
  Specificity: 0.9996

1
0


c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_950_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_950.tif:
  Dice Coefficient: 0.0217
  IoU: 0.4999
  Pixel Accuracy: 0.9998
  Precision: 0.5000
  Recall: 0.4999
  F1 Score: 0.5000
  Specificity: 0.9998

1
0
Metrics for NDWI_Mask_955_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_955.tif:
  Dice Coefficient: 0.0101
  IoU: 0.4998
  Pixel Accuracy: 0.9996
  Precision: 0.5000
  Recall: 0.4998
  F1 Score: 0.4999
  Specificity: 0.9996

1
0
Metrics for NDWI_Mask_957_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_957.tif:
  Dice Coefficient: 0.0039
  IoU: 0.4995
  Pixel Accuracy: 0.9990
  Precision: 0.4997
  Recall: 0.4998
  F1 Score: 0.4998
  Specificity: 0.9996

1
0
Metrics for NDWI_Mask_960_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_22_with_diff_kernels\dense_960.tif:
  Dice Coefficient: 0.0101
  IoU: 0.4998
  Pixel Accuracy: 0.9996
  Precision: 0.5000
  R

In [18]:
print("Average metrics:")
print(f"  Average Dice Coefficient: {average_dice:.4f}")
print(f"  Average IoU: {average_iou:.4f}")
print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Average Precision: {average_precision:.4f}")
print(f"  Average Recall: {average_recall:.4f}")
print(f"  Average F1 Score: {average_f1:.4f}")
print(f"  Average Specificity: {average_specificity:.4f}")

# Model's overall accuracy
print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

Average metrics:
  Average Dice Coefficient: 0.0541
  Average IoU: 0.4150
  Average Pixel Accuracy: 0.8045
  Average Precision: 0.7425
  Average Recall: 0.5134
  Average F1 Score: 0.4642
  Average Specificity: 0.9988
Model's Overall Accuracy: 0.8045
